In [3]:
import pandas as pd
import numpy as np
import datetime as dt

Carregar dados da camada Bronze

In [4]:
df = pd.read_csv('c:/Users/rafae/git/pipeline/bronze/twitter_sentment.csv')
print(f"Dados originais: {df.shape}")


Dados originais: (1599999, 8)


TRANSFORMAÇÃO 1: Remover Duplicatas

In [5]:
linhas_antes = len(df)
df = df.drop_duplicates()
linhas_depois = len(df)
print(f"Duplicatas removidas: {linhas_antes - linhas_depois}")


Duplicatas removidas: 0


 TRANSFORMAÇÃO 2: Tratar Valores Nulos


In [6]:
# Analisar valores nulos

print("\nValores nulos por coluna:")
print(df.isnull().sum())



Valores nulos por coluna:
0                                                                                                                      0
1467810369                                                                                                             0
Mon Apr 06 22:19:45 PDT 2009                                                                                           0
NO_QUERY                                                                                                               0
_TheSpecialOne_                                                                                                        0
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    0
data_ingestao                                                                                                          0
fonte_arquivo                                                                                                          0
dtype

In [7]:
# Exemplo: preencher valores numéricos com a mediana
colunas_numericas = df.select_dtypes(include=[np.number]).columns
for coluna in colunas_numericas:
    if df[coluna].isnull().any():
        mediana = df[coluna].median()
        df[coluna].fillna(mediana, inplace=True)
        print(f"Coluna {coluna}: preenchida com mediana = {mediana}")

# Exemplo: preencher valores categóricos com 'Desconhecido'
colunas_texto = df.select_dtypes(include=['object']).columns
for coluna in colunas_texto:
    if df[coluna].isnull().any():
        df[coluna].fillna('Desconhecido', inplace=True)
        print(f"Coluna {coluna}: preenchida com 'Desconhecido'")

TRANSFORMAÇÃO 3: Converter Tipos de Dados

In [8]:
# Exemplo: converter colunas de data
df = df.rename(columns={'Mon Apr 06 22:19:45 PDT 2009': 'data'})
df['data'] = pd.to_datetime(df['data'])

# Exemplo: converter para categorias
df = df.rename(columns={'_TheSpecialOne_': 'usuario'})
df['usuario'] = df['usuario'].astype('category')



C:\Users\rafae\AppData\Local\Temp\ipykernel_6860\1910754649.py:3: FutureWarning: Parsed string "Mon Apr 06 22:19:49 PDT 2009" included an un-recognized timezone "PDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df['data'] = pd.to_datetime(df['data'])


TRANSFORMAÇÃO 4: Padronizar Valores


In [9]:
# Exemplo: padronizar texto (remover espaços, colocar em maiúsculo)
# Renomear colunas
df = df.rename(columns={
    '0': 'indice',
    '1467810369': 'user_id',
    'Mon Apr 06 22:19:45 PDT 2009': 'data',
    'NO_QUERY': 'consulta',
    '_TheSpecialOne_': 'usuario',
    '@switchfoot [http://twitpic.com/2y1zl](http://twitpic.com/2y1zl) - Awww, that\'s a bummer.  You shoulda got David Carr of Third Day to do it. ;D': 'texto',
    'data_ingestao': 'data_ingestao',
    'fonte_arquivo': 'fonte_arquivo'
})

# Padronizar colunas de texto (remover espaços e deixar maiúsculo)
for coluna in ['usuario', 'texto', 'fonte_arquivo']:
    if coluna in df.columns:
        df[coluna] = df[coluna].str.strip().str.upper()



 VALIDAÇÕES BÁSICAS


In [10]:
print("\n=== VALIDAÇÕES DE QUALIDADE ===")

# Completude
total_celulas = df.shape[0] * df.shape[1]
celulas_preenchidas = df.count().sum()
completude = (celulas_preenchidas / total_celulas) * 100
print(f"Completude: {completude:.2f}%")

# Unicidade
duplicatas = df.duplicated().sum()
unicidade = ((len(df) - duplicatas) / len(df)) * 100
print(f"Unicidade: {unicidade:.2f}%")


=== VALIDAÇÕES DE QUALIDADE ===
Completude: 100.00%
Unicidade: 100.00%


In [11]:
# Adicionar informações de processamento

df['data_processamento'] = dt.datetime.now()


# Salvar na camada Silver

df.to_csv('c:/Users/rafae/git/pipeline/silver/twitter_sentment.csv', index=False)
print(f"/nDados limpos salvos: {df.shape}")

/nDados limpos salvos: (1599999, 9)


In [16]:
df = df.rename(columns={
    "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D": 'texto'
})
print(df.columns)

Index(['indice', 'user_id', 'data', 'consulta', 'usuario', 'texto',
       'data_ingestao', 'fonte_arquivo', 'data_processamento'],
      dtype='object')
